In [20]:
import numpy as np
from copy import deepcopy

class Mancala:
    def __init__(self, holes=7, stones=7, board=None):
        self.n_holes = holes
        self.n_stones = stones
        self.north_store = self.n_holes
        self.south_store = self.n_holes * 2 + 1
        self.board = board
        self.not_first_play = False
        if self.board is None:
            self.reset()
    
    # make a board of initial state
    def reset(self):
        self.board = np.full((self.n_holes+1) * 2, self.n_stones)
        self.board[self.n_holes] = 0
        self.board[-1] = 0
    
    # select a hole to move by player
    def step(self, side, hole):
        if not self.has_legal_move(side):
            return self.check_win()
        pos = self._get_board_pos(side, hole)
        stones = self.board[pos]
        self.board[pos] = 0
        pos += 1
        # record the postion where last stone is placed
        last_pos = 0
        while stones > 0:
            if not self.is_opponent_store(side, pos):
                self.board[pos] += 1
                stones -= 1
                if stones == 0:
                    last_pos = pos
            # add the position
            # if position outside the board make it start from the begining of the board by %
            pos = (pos + 1) % len(self.board)
        # if last postion is in the player's scoring well and is not first play
        # instruct an extra move
        if self.is_self_store(side, last_pos) and self.not_first_play:
            return "{}:continue".format(side)
        # record first play
        if not self.not_first_play:
            self.not_first_play = True
        # if the last postition is in an self emoty hole while the according oppponent's hole
        # has stones in it, take the stones from both holes
        if self.is_self_hole(side, last_pos) and self.board[last_pos] == 1 \
            and self.board[self.get_opponent_pos(last_pos)] != 0:
            scores_stored_pos = self.get_self_store(side)
            self.board[scores_stored_pos] = self.board[scores_stored_pos]+ self.board[last_pos] \
                + self.board[self.get_opponent_pos(last_pos)]
            self.board[last_pos] = 0
            self.board[self.get_opponent_pos(last_pos)] = 0
        # if the player has won, print out the message
        if self.has_won(side):
            return "{}:has won".format(side)
        return self.get_opponent_side(side)
    
    # return the position of player's scoring well
    def get_self_store(self,side):
        if side == 'south':
            return self.south_store
        else:
            return self.north_store
    
    # swap the player after each normal round
    def get_opponent_side(self,side):
        if side == 'north':
            return 'south'
        else:
            return 'north'
    
    # verify whether it is player's hole
    def is_self_hole(self, side, pos):
        return side == 'south' and self.north_store < pos < self.south_store \
            or side == 'north' and pos < self.south_store
    
    # get the according opponent hole postion given the self hole postition
    def get_opponent_pos(self, pos):
        return 2 * self.n_holes - pos
        
    # verifiy whether is player's scoring well
    def is_self_store(self,side, pos):
        return side == 'south' and pos == self.south_store \
            or side == 'north' and pos == self.north_store
    
    # verifiy whether is opponent's scoring well
    def is_opponent_store(self, side, pos):
        return side == 'south' and pos == self.north_store \
            or side == 'north' and pos == self.south_store
    
    # get the position to move in the board array
    # assuming the side chosen is reasonable
    def _get_board_pos(self, side, hole):
        return hole - 1 if side == 'north' else hole + self.n_holes
    
    # swap the side for north according to pie rule
    # assuming the swap instruction is reasonable
    def north_swap_side(self, side):
        board_copy = deepcopy(self.board)
        for i in range(len(self.board)):
            board_copy[i] = self.board[(i+self.n_holes+1)%len(self.board)]
        self.board = board_copy
        return self.get_opponent_side(side)
    
    # check whether the player has won
    def has_won(self, side):
        store_pos = self.get_self_store(side)
        if self.board[store_pos] > 49:
            return True
        return False
    
    # check whether the player has legal move
    def has_legal_move(self, side):
        if side == "north":
            offset = 0
        else:
            offset = self.n_holes+1
        for i in range (self.n_holes):
            if self.board[i+offset] > 0:
                return True
        return False
    
    # when one player does not have legal move
    # check the board to see who has won
    def check_win(self):
        north_total = self.board[self.get_self_store("north")]
        south_total = self.board[self.get_self_store("south")]
        offset = self.n_holes+1
        for i in range (self.n_holes):
            north_total += self.board[i]
            self.board[i] = 0
        self.board[self.get_self_store("north")] = north_total
        for i in range (self.n_holes):
            south_total += self.board[i+offset]
            self.board[i+offset] = 0
        self.board[self.get_self_store("south")] = south_total
        if north_total > south_total:
            return "north:has won"
        else:
            return "south:has won" 
        
    # print out of the board
    def __str__(self):
        formatter = {'int': lambda x: f'{x: >3d}'}
        return '{:2d}  {}\n    {}  {}'.format(
            self.board[self.north_store],
            np.array2string(self.board[0:self.north_store][::-1], formatter=formatter),
            np.array2string(self.board[self.north_store+1:self.south_store], formatter=formatter),
            self.board[self.south_store]
        )

game = Mancala()
print(game)
print(game.step('south', 1))
print(game)
print(game.north_swap_side('north'))
print(game)
#print(game.board)
print(game.step('south', 1))
print(game)
print(game.step('south', 2))
print(game)
print(game.step('north', 2))
print(game)
print(game.step('south', 2))
print(game)
print(game.step('north', 1))
print(game)
print(game.step('south', 1))
# game.step('south', 1)
print(game)
print(game.step('north', 7))
print(game)
print(game.step('south',3))
print(game)
print(game.step('north',7))
print(game)
print(game.step('north',5))
print(game)
print(game.step('south',7))
print(game)
print(game.step('north',7))
print(game)
print(game.step('north',6))
print(game)
print(game.step('north',7))
print(game)
print(game.step('north',4))
print(game)
print(game.step('south',6))
print(game)
print(game.step('north',7))
print(game)
print(game.step('north',6))
print(game)
print(game.step('south',5))
print(game)
print(game.step('north',7))
print(game)
print(game.step('north',5))
print(game)
print(game.step('south',7))
print(game)
print(game.step('north',6))
print(game)
print(game.step('north',7))
print(game)

# one ending
# south 不讲武德
print(game.step('north',3))
print(game)
print(game.step('south',6))
print(game)
print(game.step('south',7))
print(game)
print(game.step('south',6))
print(game)
print(game.step('south',5))
print(game)
print(game.step('south',3))
print(game)
print(game.step('south',2))
print(game)
print(game.step('north',5))
print(game)
print(game.step('north',4))
print(game)
print(game.step('north',5))
print(game)
print(game.step('south',1))
print(game)
print(game.step('north',7))
print(game)
print(game.step('north',2))
print(game)



# another ending
# 标准结局
# print(game.step('north',4))
# print(game)
# print(game.step('south',4))
# print(game)
# print("----------------------------------")
# print(game.step('north',7))
# print(game)
# print(game.step('north',3))
# print(game)
# print(game.step('south',7))
# print(game)
# print(game.step('north',7))
# print(game)
# print(game.step('north',6))
# print(game)
# print(game.step('north',7))
# print(game)
# print(game.step('north',5))
# print(game)

 0  [  7   7   7   7   7   7   7]
    [  7   7   7   7   7   7   7]  0
north
 0  [  7   7   7   7   7   7   7]
    [  0   8   8   8   8   8   8]  1
south
 1  [  8   8   8   8   8   8   0]
    [  7   7   7   7   7   7   7]  0
south:continue
 1  [  8   8   8   8   8   8   0]
    [  0   8   8   8   8   8   8]  1
north
 1  [  8   8   8   8   8   9   1]
    [  0   0   9   9   9   9   9]  2
south
 2  [  9   9   9   9   9   0   1]
    [  1   1  10   9   9   9   9]  2
north
 2  [  9   9   9   9   9   0   1]
    [  1   0  11   9   9   9   9]  2
south
12  [  9   9   9   9   9   0   0]
    [  1   0  11   9   9   0   9]  2
north
12  [  9   0   9   9   9   0   0]
    [  0   0  11   9   9   0   9]  12
south
24  [  0   0   9   9   9   0   0]
    [  1   1  12  10  10   1   0]  12
north
24  [  1   1  10  10  10   1   1]
    [  1   1   0  11  11   2   1]  13
north:continue
25  [  0   1  10  10  10   1   1]
    [  1   1   0  11  11   2   1]  13
south
26  [  1   2   0  10  10   1   1]
    [  2   2   1  12